%pip install -r requirments.txt --force

https://colab.research.google.com/drive/1IbRSNeSAZBm_6oszKSRzxABp-PbF9rTF#scrollTo=USsJS0U-LilJ

In [1]:
from flask_socketio import SocketIO, emit, join_room, leave_room
from flask import *
from flask_login import *
from flask_session import Session
from pypdf import PdfReader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from langchain_community.vectorstores import FAISS
# from datetime import timedelta
from langchain.chat_models import ChatOllama
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_core.prompts.prompt import PromptTemplate
import torch

In [2]:
app = Flask(__name__)
app.secret_key = "mkqw2o0@#mk12!mk3"
host = 'localhost'
port = 5100

socketio = SocketIO(app)
lm = LoginManager()
lm.init_app(app)

device = "cuda" if torch.cuda.is_available() else "cpu"

embedding_model = HuggingFaceEmbeddings(
    model_name="BAAI/bge-m3",
    model_kwargs = {'device': device}, # 모델이 CPU에서 실행되도록 설정. GPU를 사용할 수 있는 환경이라면 'cuda'로 설정할 수도 있음
    encode_kwargs = {'normalize_embeddings': True}, # 임베딩 정규화. 모든 벡터가 같은 범위의 값을 갖도록 함. 유사도 계산 시 일관성을 높여줌
)

ngrok = ''
if ngrok != '':
    #Meta-Llama-3.1-8B-Instruct-Q4_K_M.gguf
    #https://huggingface.co/lmstudio-community/Meta-Llama-3.1-8B-Instruct-GGUF/tree/main
    llm_model = ChatOllama(
        model='meta-llama-3.1',
        num_predict=256,
        base_url=ngrok
    )
else:
    llm_model = ChatOllama(
        model='meta-llama-3.1',
        num_predict=256,
    )

prompt_template = '''Use the following pieces of context to answer the question at the end.
If you don't find the answer in context, just say that '내용을 확인할 수 없습니다.', don't try to make up an answer.
If you find the answer in context, answer me only use korean.

context: {context}

Question: {question}
Helpful Answer:'''
rag_prompt = PromptTemplate.from_template(prompt_template)

@lm.user_loader
def user_loader(userId):
    userInfo = User.get_user_info(userId)
    return User(userInfo)

@lm.unauthorized_handler
def unauthorized():
    return redirect('/')

#region CLASS

class User(UserMixin):
    def __init__(self, info):
        self.info = info
    
    #region getter

    def get_id(self):
        return self.info['userId']

    #endregion
    
    @staticmethod
    def get_user_info(userId):
        return {'userId': userId}

#endregion CLASS

#region route

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/chatroom', methods = ['post'])
def chatroom():
    params = request.form
    userId = params['userId']
    roomId = params['roomId']
    
    session['userId'] = userId
    session['roomId'] = roomId
    
    return render_template(
        'chat.html',
        userId = userId,
        roomId = roomId
    )

@app.route('/chatbot')
def chatbot():
    if 'processed_pdf' in session:
        return 'test msg'
    return render_template('RAG.html')

@app.route('/login', methods = ['post'])
def login():
    params = request.get_json()
    userId = params['userId']
    
    userInfo = User.get_user_info(userId)
    login_user(User(userInfo))
    return jsonify({'result': 1})

@app.route('/logout')
def logout():
    logout_user()
    return redirect('/')

@app.route('/pdfprocess', methods=['post'])
def pdfprocess():
    files = [request.files[i] for i in request.files]
    
    text_sum = ''
    
    for file in files:
        reader = PdfReader(file)
        for page in reader.pages: #페이지 별로 텍스트 추출
            text = page.extract_text()
            corrected_text = text.encode('utf-8', errors='ignore').decode('utf-8') #인코딩 오류 무시 및 텍스트 누적
            text_sum += corrected_text +'\n'

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    splits = text_splitter.split_text(text_sum)
    vectorstore = FAISS.from_texts(
        splits,
        embedding = embedding_model
    )
    
    vectorstore.save_local('test')
    return jsonify({'result': 1})

@app.route('/question', methods=['POST'])
def question():
    vectorstore = FAISS.load_local(
        'test',
        embedding_model,
        allow_dangerous_deserialization=True,
    )
    params = request.get_json()
    question = params['question']
    
    memory = ConversationBufferMemory(
        memory_key='chat_history',
        return_messages=True,
    )

    conversation_chain = ConversationalRetrievalChain.from_llm(
        llm=llm_model,
        retriever=vectorstore.as_retriever(),
        condense_question_prompt=rag_prompt,
        memory=memory,
    )
    res = conversation_chain({'question': question})
    # print(res['chat_history'][1].content)
    return jsonify({'result': res['chat_history'][1].content})

#endregion route

#region socket

@socketio.on('joined', namespace = '/chatroom')
def chat_joined(d):
    roomId = session.get('roomId')
    join_room(roomId)

@socketio.on('msg', namespace = '/chatroom')
def socket_msg(d):
    roomId = session.get('roomId')
    socketio.emit('msg', d, namespace = '/chatroom', room = roomId)

#endregion socket

if __name__ == '__main__':
    print(f'http://{host}:{port}')
    socketio.run(app = app, host = host, port = port, allow_unsafe_werkzeug = True)

c:\Users\SMHRD\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
c:\Users\SMHRD\anaconda3\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


http://localhost:5100


c:\Users\SMHRD\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
